# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fae588aea90>
├── 'a' --> tensor([[ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243]])
└── 'x' --> <FastTreeValue 0x7fae588ae400>
    └── 'c' --> tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                        [-0.6758, -0.2587,  0.5419,  0.6756],
                        [ 0.2984,  0.7498,  1.3383, -0.1892]])

In [4]:
t.a

tensor([[ 0.9030, -0.1085,  0.3087],
        [ 0.5071,  1.1900, -1.5243]])

In [5]:
%timeit t.a

63.1 ns ± 0.0999 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fae588aea90>
├── 'a' --> tensor([[-1.3809, -1.2515, -0.8629],
│                   [-2.4512,  0.0605,  0.2655]])
└── 'x' --> <FastTreeValue 0x7fae588ae400>
    └── 'c' --> tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                        [-0.6758, -0.2587,  0.5419,  0.6756],
                        [ 0.2984,  0.7498,  1.3383, -0.1892]])

In [7]:
%timeit t.a = new_value

68.8 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9030, -0.1085,  0.3087],
               [ 0.5071,  1.1900, -1.5243]]),
    x: Batch(
           c: tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9030, -0.1085,  0.3087],
        [ 0.5071,  1.1900, -1.5243]])

In [11]:
%timeit b.a

63.2 ns ± 0.013 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5371,  1.2972,  2.1304],
               [-1.2785, -0.8327, -0.9435]]),
    x: Batch(
           c: tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.891 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

839 ns ± 0.884 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 14 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fae5c938c70>
├── 'a' --> tensor([[[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]],
│           
│                   [[ 0.9030, -0.1085,  0.3087],
│                    [ 0.5071,  1.1900, -1.5243]]])
└── 'x' --> <FastTreeValue 0x7fad84a9ebb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fae481c3b50>
├── 'a' --> tensor([[ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243],
│                   [ 0.9030, -0.1085,  0.3087],
│                   [ 0.5071,  1.1900, -1.5243]])
└── 'x' --> <FastTreeValue 0x7fae481c38e0>
    └── 'c' --> tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                        [-0.6758, -0.2587,  0.5419,  0.6756],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.3 µs ± 67.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3115,  0.1401, -0.3194, -1.0053],
                       [-0.6758, -0.2587,  0.5419,  0.6756],
                       [ 0.2984,  0.7498,  1.3383, -0.1892]],
              
                      [[-1.3115,  0.1401, -0.3194, -1.0053],
                       [-0.6758, -0.2587,  0.5419,  0.6756],
                       [ 0.2984,  0.7498,  1.3383, -0.1892]],
              
                      [[-1.3115,  0.1401, -0.3194, -1.0053],
                       [-0.6758, -0.2587,  0.5419,  0.6756],
                       [ 0.2984,  0.7498,  1.3383, -0.1892]],
              
                      [[-1.3115,  0.1401, -0.3194, -1.0053],
                       [-0.6758, -0.2587,  0.5419,  0.6756],
                       [ 0.2984,  0.7498,  1.3383, -0.1892]],
              
                      [[-1.3115,  0.1401, -0.3194, -1.0053],
                       [-0.6758, -0.2587,  0.5419,  0.6756],
                       [ 0.2984,  0.7498,  1.3383, -0.1892]],

In [26]:
%timeit Batch.stack(batches)

79 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892],
                      [-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892],
                      [-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892],
                      [-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892],
                      [-1.3115,  0.1401, -0.3194, -1.0053],
                      [-0.6758, -0.2587,  0.5419,  0.6756],
                      [ 0.2984,  0.7498,  1.3383, -0.1892],
                      [-1.3115,  0.1401, -0.3194, -1.0053],
                   

In [28]:
%timeit Batch.cat(batches)

149 µs ± 368 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

342 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
